# Prepare Data

In [1]:
import pandas as pd

concepts_df = pd.read_csv('../lib/data/raw/vocabularies/CONCEPT.csv', sep=',')


/var/folders/d4/zjykh2gs0ddchpt100y0llr40000gp/T/ipykernel_35919/772158266.py:3: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  concepts_df = pd.read_csv('../lib/data/raw/vocabularies/CONCEPT.csv', sep=',')


In [2]:
concepts_df

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
8640324,2213270,Molecular cytogenetics; chromosomal in situ hy...,Measurement,CPT4,CPT4,S,88272,19700101,20991231,NaN
8640325,45889102,"Bone graft, any donor area",Procedure,CPT4,CPT4 Hierarchy,C,1003752,20141010,20991231,NaN
8640326,2213271,Molecular cytogenetics; chromosomal in situ hy...,Measurement,CPT4,CPT4,S,88273,19700101,20991231,NaN
8640327,2213272,Molecular cytogenetics; interphase in situ hyb...,Measurement,CPT4,CPT4,S,88274,19700101,20991231,NaN


In [3]:
concepts_df.rename({'concept_name': 'target'}, axis=1, inplace=True)

In [4]:
data_dict = concepts_df[["concept_id", "target"]].to_dict(orient='records')

# Create Schema

In [6]:
import typesense

client = typesense.Client(
    {
        "api_key": "xyz",
        "nodes": [{"host": "localhost", "port": "8108", "protocol": "http"}],
        "connection_timeout_seconds": 10,
    }
)

try:
    client.collections["concepts"].delete()
except Exception as e:
    pass


schema = {
    "name": "targets",
    "fields": [
        {"name": "concept_id", "type": "int32"},
        {"name": "target", "type": "string"},
        {
            "name": "embedding",
            "type": "float[]",
            "embed": { # auto embed targets
                "from": ["target"],
                "model_config": {"model_name": "ts/multilingual-e5-small"},
                "indexing_prefix": "query:",
                "query_prefix": "query:"
            },
        },
    ],
    "default_sorting_field": "concept_id",
}


create_response = client.collections.create(schema)
print(create_response)

{'created_at': 1711131364, 'default_sorting_field': 'concept_id', 'enable_nested_fields': False, 'fields': [{'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'concept_id', 'optional': False, 'sort': True, 'type': 'int32'}, {'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'target', 'optional': False, 'sort': False, 'type': 'string'}, {'embed': {'from': ['target'], 'indexing_prefix': 'query:', 'model_config': {'model_name': 'ts/multilingual-e5-small'}, 'query_prefix': 'query:'}, 'facet': False, 'index': True, 'infix': False, 'locale': '', 'name': 'embedding', 'num_dim': 384, 'optional': False, 'sort': False, 'type': 'float[]'}], 'name': 'targets', 'num_documents': 0, 'symbols_to_index': [], 'token_separators': []}


# Insert in Typesense and encode

In [7]:
import typesense

client = typesense.Client({
    'api_key': 'xyz',
    'nodes': [{
        'host': 'localhost',
        'port': '8108',
        'protocol': 'http'
    }],
    'connection_timeout_seconds': 100
})

client.collections['targets'].documents.import_(data_dict[:5000], {'action': 'upsert'}, batch_size=10000)

[{'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},


# Query

In [8]:
data_dict[700]

{'concept_id': 2995339,
 'target': 'Antimicrobial Hand Gel (Elyptol) 70% Gel, 60 Ml'}

In [9]:
query = "Gel antimicrobiano 60 mililitros"

search_parameters = {
'q'                          : query,
'query_by'                   : 'embedding'
}

In [10]:

results = client.collections['targets'].documents.search(search_parameters)

In [11]:
matches = [i['document']['concept_id'] for i in results['hits']]

In [12]:
concepts_df.loc[concepts_df['concept_id'].isin(matches)]

,concept_id,target,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
699,2997836,Antimicrobial Hand Gel (Elyptol) 70% Gel,Device,AMT,Trade Product Unit,S,1006421000168107,20161101,20991231,NaN
700,2995339,"Antimicrobial Hand Gel (Elyptol) 70% Gel, 60 Ml",Device,AMT,Trade Product Pack,S,1006431000168105,20161101,20991231,NaN
701,2996298,"Antimicrobial Hand Gel (Elyptol) 70% Gel, 60 M...",Device,AMT,Containered Pack,S,1006441000168101,20161101,20991231,NaN
704,2995872,"Antimicrobial Hand Gel (Elyptol) 70% Gel, 500 Ml",Device,AMT,Trade Product Pack,S,1006471000168108,20161101,20991231,NaN
2094,2976638,Orlistat 60 Mg Capsule,Drug,AMT,Med Product Unit,NaN,1021021000168100,20161101,20991231,NaN
2095,2939390,Alli 60 Mg Capsule,Drug,AMT,Trade Product Unit,NaN,1021031000168102,20161101,20991231,NaN
2285,2957387,"Epiduo Forte Gel, 60 G",Drug,AMT,Trade Product Pack,NaN,1022811000168109,20161101,20991231,NaN
3451,2991024,Tolvaptan 60 Mg Tablet,Drug,AMT,Med Product Unit,NaN,1036181000168108,20161101,20991231,NaN
3452,2965550,Jinarc 60 Mg Tablet,Drug,AMT,Trade Product Unit,NaN,1036191000168106,20161101,20991231,NaN
4266,2994361,Ziprox 60 Mg Capsule,Drug,AMT,Trade Product Unit,NaN,1044441000168100,20161101,20991231,NaN


# With test data

In [13]:
client.collections['targets'].documents.delete({'filter_by': 'concept_id: >=0'})


{'num_deleted': 5000}

In [14]:
import pickle

with open('../lib/artifacts/dicts/sources.pickle', 'rb') as handle:
    sources_dict = pickle.load(handle)
with open('../lib/artifacts/dicts/targets.pickle', 'rb') as handle:
    targets_dict = pickle.load(handle)

In [15]:
target_dict = [{"concept_id": k, "target": v} for k, v in targets_dict.items()]

In [16]:
client.collections['targets'].documents.import_(target_dict, {'action': 'upsert'}, batch_size=10000)

[{'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},
 {'success': True},


In [17]:
total = len(sources_dict)
top1=0
top5=0
top10=0

for i in range(total):
    query = sources_dict[i]

    search_parameters = {
    'q'                          : query,
    'query_by'                   : 'embedding'
    }

    results = client.collections['targets'].documents.search(search_parameters)
    matches = [i['document']['concept_id'] for i in results['hits']]
    if i == matches[0]:
        top1+=1
    if i in matches[:5]:
        top5+=1
    if i in matches[:10]:
        top10+=1

In [18]:
print(f"top1: {top1/total:.2%}, top5: {top5/total:.2%}, top10: {top10/total:.2%}")

top1: 43.20%, top5: 62.56%, top10: 66.88%
